In [1]:
!pip install openpyxl

In [3]:
!pip install kiwipiepy

     ---------------------------------------- 0.0/79.5 MB ? eta -:--:--
     --- ------------------------------------ 6.3/79.5 MB 87.8 MB/s eta 0:00:01
     ---- ----------------------------------- 8.9/79.5 MB 30.5 MB/s eta 0:00:03
     ------------- ------------------------- 28.0/79.5 MB 56.6 MB/s eta 0:00:01
     --------------- ----------------------- 31.7/79.5 MB 50.3 MB/s eta 0:00:01
     ------------------ -------------------- 38.5/79.5 MB 42.7 MB/s eta 0:00:01
     ------------------------- ------------- 52.4/79.5 MB 47.3 MB/s eta 0:00:01
     -------------------------------- ------ 65.8/79.5 MB 50.3 MB/s eta 0:00:01
     ------------------------------------- - 76.0/79.5 MB 50.2 MB/s eta 0:00:01
     --------------------------------------  79.4/79.5 MB 49.7 MB/s eta 0:00:01
     ---------------------------------------- 79.5/79.5 MB 43.9 MB/s  0:00:01
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to bu

In [32]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
from kiwipiepy import Kiwi 
from collections import Counter
import re


# ==== 키워드 합계 & 사이즈 =======

src = r"naver_keyword.xlsx"   # r = '.' 온점이 문자 인식이 안될 수 있기 때문에 온전히 그대로 문자로 인식하라고 쓸때 앞에 r 붙임
out = r"naver_keyword_result_v2.xlsx"   # 새로운 이름의 파일로 결과 저장 


def find_col_by_header(ws, keywords, header_row = 1) :    # def 선언하다 여기선 함수 선언 / 
    # header_row 대신 1만 넣어도 되나 나중에 1이 어떤건지 알아볼수 없으므로 저렇게 값을 적어줌 
    # keywords = ["연관키워드", "월간검색수(pc)", "월감검색수(moblie)",..., "월평균노출"]
   
    for col in range(1, ws.max_column + 1) : 
        v = ws.cell(row=header_row, column=col).value 
        if not v : # if not 조건 값이 없을때만 실행해라  if not + continue = 조건값이 없으면 건너뛰고 그대로 실행 
            continue 
        s= str(v).replace(" ", "") # 공백없애기  월간검색수  (pc) ->월간검색수(pc)
        ok = True
        for kw in  keywords :
            if kw.replace(" ","")  not in s :
                ok = False
                break  # 반복문 종료 
        if ok : 
            return col
    return None

wb = load_workbook(src)
ws = wb.active

pc_col = (find_col_by_header(ws, ["월간검색수", "PC"]) or find_col_by_header(ws, ["월간검색수", "피씨"]) or 2) 
pc_col = (find_col_by_header(ws, ["월간검색수", "모바일"]) or find_col_by_header(ws, ["월간검색수", "Mobile"]) or 3) 

total_col = 4 
size_col = 5 

ws.cell(row = 1, column=total_col).value = "전체검색량(PC+모바일)" # cell 함수 셀 안에 있는 값 추출
ws.cell(row = 1, column=total_col).value = "키워드규모(전체검색량)" 

for c in [total_col, size_col] :
    cell = ws.cell(row=1, column=c)
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

def to_int(x) :
    if x is None : 
        return 0 
    if isinstance(x, (int, float)) : # isinstance 함수 : a 이거나 b면 
        return int(x)
    s = str(x).strip()
    if s in ("<10", "< 10") :
        return 0
    s = s.replace(",", "")
    try :  # 오류가 날 것 같은 코드를 try 블록에 넣고, 오류가 났을 때의 대처법을 except 블록에 적음 
        return int(float(s))  # "145.75" => int() 에러. 실수인 문자-> 정수로 바로 못바꿈 / int(float()) -> 146  문자-실수-정수로 변경하는 과정 필요 
    except :
        return 0 


for r in range(2, ws.max_row + 1) :
    pc = to_int(ws.cell(row=r, column=pc_col).value)
    mo = to_int(ws.cell(row=r, column=pc_col).value)
    total = pc + mo 
    ws.cell(row=r, column=total_col).value = total 

    if total >= 3000 : 
        size = "대형(3000+)"
    elif total >= 1000 :
        size = "중형(1000+)"
    else :
        size = "소형(<1000)" 
    ws.cell(row=r, column=size_col).value = size

ws.auto_filter.ref = f"A1:{ws.cell(row=1, column=max(ws.max_column, size_col)).coordinate}" 




# ======= 키워드 형태소 분석 ===========

fixed_category = {
    "원터치": ("타입", "원터치"),
    "드롭": ("타입", "드롭"),
    "클립": ("타입", "클립"),
    "명품": ("브랜드", "명품"),
    "브랜드": ("브랜드", "브랜드"),
    "여자": ("성별", "여자"),
}

brands = {"샤넬", "스톤핸즈", "스와로브스키", "티파니", "까르띠에", "불가리", "반클리프", "구찌"}

def clssify_token(token: str) :
    t = str(token).strip()

    if not t : 
        return("기타", "")
    if t in fixed_category : 
        return fixed_category[t]
    if t in brands : 
        return("브랜드", t)
    return("기타", "")
    


if "키워드_빈도" in wb.sheetnames :
    del wb["키워드_빈도"]
ws_kw = wb.create_sheet("키워드_빈도") 

keyword_col = 1 
text = [
    row[0] for row in ws.iter_rows(min_row=2, max_col=1, values_only=True)   # iter_rows : 순환하면서 실행해라
    if row and row[0]
] 
all_text = " ".join(map(str, text)) 
# ["a","b", "c"] => (a b c ) 각각의 문자가 아닌 띄어쓰기를 기준으로 긴 문자모음 문장으로 만듦
# print(all_text)


kiwi = Kiwi()    # 키위에서는 단어/문자를 토큰으로 지칭 
tokens = kiwi.tokenize(all_text)
# [{"tag": "N명사", "from": "귀걸이"}, {...}]


noun_tags = {"NNG", "NNP", "NNB", "NR"}
noun_tokens = [t.form for t in tokens if t.tag in noun_tags ]

# NNG= Noun Noun General 일반명사 
# NNP = Noun Noun Proper 고유명사 (이름, 브랜드 명칭 등등 ) 
# NNB = Noun Noun Bound : 의존명사 / 수를 세는 명사 (몇 개, 몇 명, 몇 번, 몇 가지 ) 
# NR = Noun Mumeral : 수사 (하나 둘 셋 열 .. 



# ===== 정규표현식 ==========

# GA4, BigQuery 에서 사용 
# 탐색 보고서/  겁색창에서 검색하려고 할때 / 특정 경로값을 포함한 주소를 찾아야 할때 


# findall( , "찾아와야할 위치") 
regex_tokens = re.findall(r"[A-Xa-x]*\d+[A-Xa-x]+|\d+[A-Xa-x]*", all_text)  
# [범위 지정]  
# * = 나올수도 안나올 수도 있다는 표기  
# + = 1개 이상 있음  
# +| = 추가로  

stopwords = {"엑스", "구매", "다음", "때문"}  # 불용어 
valid_one = {"금", "은", "동", "침", "링"} # 1글자이지만 중요 키워드
combined = noun_tokens + regex_tokens
filtered = [
    t for t in combined 
    if (len(str(t)) > 1 or t in valid_one) 
    and t not in stopwords
]

count = Counter(filtered)
print(count)

ws_kw.append(["키워드", "빈도", "카테고리", "세부범주"])
for cell in ws_kw[1] : 
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

for kw, frep in count.most_common() :
    cat, sub = clssify_token(kw)
    ws_kw.append([kw, frep, cat, sub])

ws_kw.auto_filter.re = f"A1:E{ws_kw.max_row}"
    

wb.save(out)
print ("done :", out)



Counter({'귀걸이': 443, '링': 79, '14K': 74, '원터치': 35, '실버': 29, '18K': 27, '금': 23, '여자': 19, '드롭': 19, '골드': 19, '브랜드': 17, '큐빅': 16, '은': 15, '침': 14, '데일리': 12, '진주': 11, '이어': 11, '추천': 11, '여성': 11, '악세사리': 10, '미니': 8, '20': 8, '목걸이': 7, '선물': 7, '쇼핑몰': 7, '하트': 7, '925': 7, '귀고리': 6, '로즈': 6, '클립': 6, '주얼리': 6, '40': 6, '리본': 5, '심플': 5, '30': 5, '명품': 4, '화이트': 4, '하트 귀걸이': 4, '체인': 4, '블랙': 4, '사각': 4, '14': 4, '50': 4, '진주 귀걸이': 3, '물방울': 3, '나비': 3, '클로버': 3, '가격': 3, '볼륨': 3, '액세서리': 3, '유행': 3, '모양': 3, '엄마': 3, '귓바퀴': 3, '생일': 2, '반지': 2, '스터드': 2, '패션': 2, '알러지': 2, '귀찌': 2, '도금': 2, '케이': 2, '친구': 2, '파이프': 2, '사이트': 2, '하객': 2, '컷팅': 2, '쥬얼리': 2, '편집': 2, '연예인': 2, '여름': 2, '당일': 2, '배송': 2, '민자': 2, '유니크': 2, '18': 2, '진주 목걸이': 1, '스와로브스키': 1, '커플링': 1, '로이드': 1, '팔찌': 1, '웨딩': 1, '백금': 1, '결혼식': 1, '이어커프': 1, '아이': 1, '옐로우': 1, '그림': 1, '종로': 1, '후프': 1, '실버이어커프': 1, '말발굽': 1, '홍대': 1, '웨이': 1, '겨울': 1, '비즈': 1, '신부': 1, '핑크': 1, '매듭': 1, '미러': 1, '스타일': 1, '티파니': 1, '